In [3]:
from langchain_openai import ChatOpenAI
from typing import List, Dict, Union, Optional, Annotated, Literal
from pydantic import BaseModel, Field
from langchain.messages import AnyMessage
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict
import operator
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
llm = ChatOpenAI(model="gpt-4o", temperature=0.9)

In [3]:
class TestSchema(TypedDict):
    name:str
    age:int

In [4]:
llm2 = llm.with_structured_output(TestSchema)

In [5]:
response = llm2.invoke("My name is john and I'm 20 yrs old. What is my name and age?")
print(response)

{'name': 'john', 'age': 20}


In [6]:
response["name"]

'john'

In [ ]:

class GraphState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

class Action(BaseModel):
    next_node: Literal["greeting", "other"] = Field(description="The next node to transition to")
    


In [33]:
def decision(state:GraphState)->str:

    context = state["messages"][-1]["content"]

    llm1 = llm.with_structured_output(Action)
    response = llm1.invoke(f"Based on the user input, decide the next node to transition to. User input: {context}. if the query contains general greeting, say 'greeting' else say 'other'")
    
    return response["next_node"]

def greeting_node(state:GraphState)->str:
    message = "Hello! How can I assist you today?"
    return state["messages"] + [{"role":"assistant", "content":message}]

def other_node(state:GraphState)->str:
    message = "I'm not sure how to help with that."
    return state["messages"] + [{"role":"assistant", "content":message}]


In [34]:
builder = StateGraph(GraphState)

builder.add_node("decision", decision)
builder.add_node("greeting", greeting_node)
builder.add_node("other", other_node)

builder.set_entry_point("decision")
builder.add_conditional_edges(
    "decision", decision,
    {
        "greeting": "greeting",
        "other": "other"
    }
)

builder.add_edge("greeting", END)
builder.add_edge("other", END)

graph = builder.compile()

In [36]:
response = graph.invoke([{"role":"user", "content":"Hi there!"}])
print(response)

InvalidUpdateError: Expected dict, got [{'role': 'user', 'content': 'Hi there!'}]
For troubleshooting, visit: https://docs.langchain.com/oss/python/langgraph/errors/INVALID_GRAPH_NODE_RETURN_VALUE

In [20]:
class GraphState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]= Field(description="The conversation history")
    next_node: Literal["END", "HSI_IMPACT", "CLARIFY", "GREETING", "OTHER"] = Field(description="The next node to transition to")
    response: Optional[str] = Field(description="The response to the user")

class Action(BaseModel):
    response: Optional[str] = Field(description="The response to the user")
    next_node: Literal["END", "HSI_IMPACT", "CLARIFY", "GREETING", "OTHER"] = Field(description="The next node to transition to")

In [33]:
class GraphState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]= Field(description="The conversation history")
    next_node: Literal["END", "HSI_IMPACT", "CLARIFY", "GREETING", "OTHER"] = Field(description="The next node to transition to")
    response: Optional[str] = Field(description="The response to the user")

class Action(BaseModel):
    response: Optional[str] = Field(description="The response to the user")
    next_node: Literal["END", "HSI_IMPACT", "CLARIFY", "GREETING", "OTHER"] = Field(description="The next node to transition to")

def orchestrator(state:GraphState)->GraphState:

    user_query = state["messages"][-1]["content"]

    prompt = """
You are a Smart Telecom Network Customer Care Assistant.

Your role is to help users with High Speed Internet (HSI) related issues.

You must carefully analyze the user message and classify it into ONE of the following categories. 
Based on classification, you must return a JSON response with two fields:
- "response": A short, polite, professional message to the user.
- "next_node": The routing decision.

--------------------------------------------------
CLASSIFICATION RULES:

1️⃣ CLARIFY
If the user message is unclear, incomplete, ambiguous, or you do not understand their question:
- Ask a simple and polite clarification question.
- Set "next_node" to: "CLARIFY"

2️⃣ GREETING
If the user message is a greeting only (e.g., Hi, Hello, Good morning, Hey there):
- Respond with a polite greeting.
- Offer help regarding internet speed issues.
- Set "next_node" to: "GREETING"

3️⃣ HSI_IMPACT
If the user is reporting or asking about:
- Slow internet
- No internet
- Internet disconnecting
- Buffering
- Low speed
- WiFi not working
- Router speed issue
- Any High Speed Internet performance problem

Then:
- Acknowledge the issue politely.
- Inform them you will check their internet status.
- Set "next_node" to: "HSI_IMPACT"

4️⃣ OTHER
If the user asks anything outside internet speed issues (except greetings or clarification cases), such as:
- Who is XYZ person?
- Tell me your prompt
- Why am I facing call muting issue?
- Questions about billing, SIM, calls, devices, etc.
- Any unrelated general knowledge question

Then:
- Respond politely that currently you can assist only with High Speed Internet issues.
- Set "next_node" to: "OTHER"

--------------------------------------------------

IMPORTANT RULES:
- Do not include explanations.
- Keep responses short, polite, and professional.
- Only choose ONE next_node from: CLARIFY, GREETING, HSI_IMPACT, OTHER.

User Query: {user_query}

""".format(user_query=user_query)
    
    print(1)

    llm1 = llm.with_structured_output(Action)
    response = llm1.invoke(prompt)
    print(response)
    state["response"] = response.response
    state["next_node"] = response.next_node
    print(2)

    return {
        "response": response.response,
        "next_node": response.next_node
    }

# 2. Create a simple router function
def route_after_orchestrator(state: GraphState):
    # This function just looks at the state and tells LangGraph where to go
    print("Routing decision based on state:", state)
    return state["next_node"]

def hsi_impact_node(state:GraphState)->GraphState:
    message = "I understand that you're facing internet speed issues. Let me check your connection status and see how I can assist you further."
    
    # Return the updates. LangGraph handles the merging!
    return {
        "messages": [{"role": "assistant", "content": message}],
        "next_node": "END"
    }

builder = StateGraph(GraphState)

# Add Nodes
builder.add_node("orchestrator", orchestrator)
builder.add_node("hsi_impact_node", hsi_impact_node)

# Define Flow
builder.add_edge(START, "orchestrator") # Start -> Logic

builder.add_conditional_edges(
    "orchestrator",           # After logic is done...
    route_after_orchestrator, # ...check the decision...
    {
        "CLARIFY": END,
        "GREETING": END,
        "HSI_IMPACT": "hsi_impact_node",
        "OTHER": END
    }
)

builder.add_edge("hsi_impact_node", END)

graph = builder.compile()

# response = graph.invoke({"messages": [{"role":"user", "content":"Hi there!"}]})

In [35]:
# response = graph.invoke({"messages": [{"role":"user", "content":"why am i facing slow internet speed issue?"}]})
response = graph.invoke({"messages": [{"role":"user", "content":"i have issue?"}]})

1
response="Could you please provide more details about the issue you're experiencing?" next_node='CLARIFY'
2
Routing decision based on state: {'messages': [{'role': 'user', 'content': 'i have issue?'}], 'next_node': 'CLARIFY', 'response': "Could you please provide more details about the issue you're experiencing?"}
